In [1]:
from pyspark.sql import SparkSession
spark = (SparkSession
      .builder
      .appName("Chatper4")
      .getOrCreate())

23/04/10 16:38:51 WARN Utils: Your hostname, MacBook-Pro-7.local resolves to a loopback address: 127.0.0.1; using 10.7.133.4 instead (on interface en0)
23/04/10 16:38:51 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/10 16:38:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
# 데이터 경로
csv_file = "/Users/hyunjun/vscode/Spark/databricks-datasets/learning-spark-v2/flights/departuredelays.csv"

# 읽고 임시뷰를 생성
# 스키마 추론(더 큰 파일일 경우에는 스키마를 지정해주자)
df = (spark.read.format("csv")
      .option("inferSchema", "true")
      .option("header", "true")
      .load(csv_file))

df.createOrReplaceTempView("us_delay_flights_tbl")

In [3]:
spark.sql("""SELECT distance, origin, destination
    FROM us_delay_flights_tbl WHERE distance > 1000
    ORDER BY distance DESC""").show(10)

+--------+------+-----------+
|distance|origin|destination|
+--------+------+-----------+
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
+--------+------+-----------+
only showing top 10 rows



In [4]:
spark.sql("""SELECT delay, origin, destination,
              CASE
                  WHEN delay > 360 THEN 'Very Long Delays'
                  WHEN delay > 120 AND delay < 360 THEN 'Long Delays'
                  WHEN delay > 60 AND delay < 120 THEN 'Short Delays'
                  WHEN delay > 0 and delay < 60  THEN  'Tolerable Delays'
                  WHEN delay = 0 THEN 'No Delays'
                  ELSE 'Early'
               END AS Flight_Delays
               FROM us_delay_flights_tbl
               ORDER BY origin, delay DESC""").show(10)

+-----+------+-----------+-------------+
|delay|origin|destination|Flight_Delays|
+-----+------+-----------+-------------+
|  333|   ABE|        ATL|  Long Delays|
|  305|   ABE|        ATL|  Long Delays|
|  275|   ABE|        ATL|  Long Delays|
|  257|   ABE|        ATL|  Long Delays|
|  247|   ABE|        DTW|  Long Delays|
|  247|   ABE|        ATL|  Long Delays|
|  219|   ABE|        ORD|  Long Delays|
|  211|   ABE|        ATL|  Long Delays|
|  197|   ABE|        DTW|  Long Delays|
|  192|   ABE|        ORD|  Long Delays|
+-----+------+-----------+-------------+
only showing top 10 rows



In [5]:
# In Python
from pyspark.sql.functions import col, desc 
(df.select("distance", "origin", "destination")
      .where(col("distance") > 1000)
      .orderBy(desc("distance"))).show(10)
# Or
(df.select("distance", "origin", "destination")
    .where("distance > 1000")
    .orderBy("distance", ascending=False).show(10))

+--------+------+-----------+
|distance|origin|destination|
+--------+------+-----------+
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
+--------+------+-----------+
only showing top 10 rows

+--------+------+-----------+
|distance|origin|destination|
+--------+------+-----------+
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
+--------+------+-----------+
only showing top 10 rows



In [6]:
#// In Scala/Python
spark.sql("CREATE DATABASE learn_spark_db")
spark.sql("USE learn_spark_db")

DataFrame[]

In [7]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Chatper4").config("spark.sql.legacy.createHiveTableByDefault", "false").getOrCreate()

#// In Scala/Python
spark.sql("CREATE TABLE managed_us_delay_flights_tbl (date STRING, delay INT,distance INT, origin STRING, destination STRING)")

DataFrame[]

In [8]:
# In Python
# Path to our US flight delays CSV file
csv_file = "/Users/hyunjun/vscode/Spark/databricks-datasets/learning-spark-v2/flights/departuredelays.csv" # Schema as defined in the preceding example
schema="date STRING, delay INT, distance INT, origin STRING, destination STRING" 
flights_df = spark.read.csv(csv_file, schema=schema) 
# 위에서 만든 테이블 삭제
spark.sql("DROP TABLE IF EXISTS managed_us_delay_flights_tbl")
flights_df.write.saveAsTable("managed_us_delay_flights_tbl")

23/04/10 16:39:07 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 96.54% for 7 writers
23/04/10 16:39:07 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 84.47% for 8 writers
23/04/10 16:39:08 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 96.54% for 7 writers


In [9]:
# 위에서 만든 테이블 삭제
# spark.sql("DROP TABLE IF EXISTS us_delay_flights_tbl")

spark.sql("""CREATE TABLE us_delay_flights_tbl(date STRING, delay INT,
    distance INT, origin STRING, destination STRING
)
USING csv OPTIONS (PATH
"/Users/hyunjun/vscode/Spark/databricks-datasets/learning-spark-v2/flights/departuredelays.csv")""")

DataFrame[]

In [11]:
# 덮어쓰기
spark.conf.set("spark.sql.legacy.allowNonEmptyLocationInCTAS", "true")

spark.sql("DROP TABLE IF EXISTS us_delay_flights_tbl")

(flights_df
      .write
      .option("path", "/tmp/data/us_flights_delay")
      .saveAsTable("us_delay_flights_tbl"))

23/04/10 16:39:17 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 96.54% for 7 writers
23/04/10 16:39:17 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 84.47% for 8 writers


23/04/10 16:39:18 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 96.54% for 7 writers


In [12]:
# -- In SQL
spark.sql("""CREATE OR REPLACE GLOBAL TEMP VIEW us_origin_airport_SFO_global_tmp_view AS SELECT date, delay, origin, destination from us_delay_flights_tbl WHERE origin = 'SFO'""")
spark.sql("""CREATE OR REPLACE TEMP VIEW us_origin_airport_JFK_tmp_view AS SELECT date, delay, origin, destination from us_delay_flights_tbl WHERE origin = 'JFK'""")

DataFrame[]

In [13]:
# In Python
df_sfo = spark.sql("SELECT date, delay, origin, destination FROM us_delay_flights_tbl WHERE origin = 'SFO'")
df_jfk = spark.sql("SELECT date, delay, origin, destination FROM us_delay_flights_tbl WHERE origin = 'JFK'")
# Create a 전역 임시 and 임시 뷰
df_sfo.createOrReplaceGlobalTempView("us_origin_airport_SFO_global_tmp_view")
df_jfk.createOrReplaceTempView("us_origin_airport_JFK_tmp_view")

In [14]:
# -- In SQL
spark.sql("""SELECT * FROM global_temp.us_origin_airport_SFO_global_tmp_view""")
spark.sql("""SELECT * FROM us_origin_airport_JFK_tmp_view""")
# // In Scala/Python
spark.read.table("us_origin_airport_JFK_tmp_view")
# // Or
spark.sql("SELECT * FROM us_origin_airport_JFK_tmp_view")

DataFrame[date: string, delay: int, origin: string, destination: string]

In [15]:
# # -- In SQL
# DROP VIEW IF EXISTS us_origin_airport_SFO_global_tmp_view; 
# DROP VIEW IF EXISTS us_origin_airport_JFK_tmp_view

# // In Scala/Python
spark.catalog.dropGlobalTempView("us_origin_airport_SFO_global_tmp_view")
spark.catalog.dropTempView("us_origin_airport_JFK_tmp_view")

True

In [16]:
#  // In Scala/Python
spark.catalog.listDatabases()
spark.catalog.listTables()
spark.catalog.listColumns("us_delay_flights_tbl")

[Column(name='date', description=None, dataType='string', nullable=True, isPartition=False, isBucket=False),
 Column(name='delay', description=None, dataType='int', nullable=True, isPartition=False, isBucket=False),
 Column(name='distance', description=None, dataType='int', nullable=True, isPartition=False, isBucket=False),
 Column(name='origin', description=None, dataType='string', nullable=True, isPartition=False, isBucket=False),
 Column(name='destination', description=None, dataType='string', nullable=True, isPartition=False, isBucket=False)]

In [17]:
# # -- In SQL
# CACHE [LAZY] TABLE <table-name> 
# UNCACHE TABLE <table-name>

In [18]:
us_flights_df = spark.sql("SELECT * FROM us_delay_flights_tbl")
us_flights_df2 = spark.table("us_delay_flights_tbl")

In [19]:
# In Python
file = """/Users/hyunjun/vscode/Spark/databricks-datasets/learning-spark-v2/flights/summary-data/parquet/2010-summary.parquet/"""
df = spark.read.format("parquet").load(file)
df.show(10)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|    1|
|    United States|            Ireland|  264|
|    United States|              India|   69|
|            Egypt|      United States|   24|
|Equatorial Guinea|      United States|    1|
|    United States|          Singapore|   25|
|    United States|            Grenada|   54|
|       Costa Rica|      United States|  477|
|          Senegal|      United States|   29|
|    United States|   Marshall Islands|   44|
+-----------------+-------------------+-----+
only showing top 10 rows



In [20]:
spark.sql("""CREATE OR REPLACE TEMPORARY VIEW us_delay_flights_tbl USING parquet OPTIONS (path '/Users/hyunjun/vscode/Spark/databricks-datasets/learning-spark-v2/flights/summary-data/parquet/2010-summary.parquet/')""")

DataFrame[]

In [21]:
# -- In SQL
spark.sql("""
    CREATE OR REPLACE TEMPORARY VIEW us_delay_flights_tbl USING parquet 
    OPTIONS (
        path "/Users/hyunjun/vscode/Spark/databricks-datasets/learning-spark-v2/flights/summary-data/parquet/2010-summary.parquet/" )
""")

# In Python
spark.sql("SELECT * FROM us_delay_flights_tbl").show(10)

# # DataFrame으로 테이블을 읽어옵니다.
# us_delay_flights_df = spark.read.format("parquet").option("path", "/Users/hyunjun/vscode/Spark/databricks-datasets/learning-spark-v2/flights/summary-data/parquet/2010-summary.parquet/").load()

# # 임시 뷰를 생성합니다.
# us_delay_flights_df.createOrReplaceTempView("us_delay_flights_tbl")

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|    1|
|    United States|            Ireland|  264|
|    United States|              India|   69|
|            Egypt|      United States|   24|
|Equatorial Guinea|      United States|    1|
|    United States|          Singapore|   25|
|    United States|            Grenada|   54|
|       Costa Rica|      United States|  477|
|          Senegal|      United States|   29|
|    United States|   Marshall Islands|   44|
+-----------------+-------------------+-----+
only showing top 10 rows



In [22]:
# In Python
(df.write.format("parquet")
    .mode("overwrite")
    .option("compression", "snappy")
    .save("/tmp/data/parquet/df_parquet"))

In [23]:
# In Python
file = "/Users/hyunjun/vscode/Spark//databricks-datasets/learning-spark-v2/flights/summary-data/json/*"
df = spark.read.format("json").load(file)
df.show(10)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|   15|
|    United States|            Croatia|    1|
|    United States|            Ireland|  344|
|            Egypt|      United States|   15|
|    United States|              India|   62|
|    United States|          Singapore|    1|
|    United States|            Grenada|   62|
|       Costa Rica|      United States|  588|
|          Senegal|      United States|   40|
|          Moldova|      United States|    1|
+-----------------+-------------------+-----+
only showing top 10 rows



In [24]:
# -- In SQL
spark.sql("""
    CREATE OR REPLACE TEMPORARY VIEW us_delay_flights_tbl USING json
    OPTIONS (
    path "/Users/hyunjun/vscode/Spark/databricks-datasets/learning-spark-v2/flights/summary-data/json/*"
    )
""")

#// In Scala/Python
spark.sql("SELECT * FROM us_delay_flights_tbl").show(10)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|   15|
|    United States|            Croatia|    1|
|    United States|            Ireland|  344|
|            Egypt|      United States|   15|
|    United States|              India|   62|
|    United States|          Singapore|    1|
|    United States|            Grenada|   62|
|       Costa Rica|      United States|  588|
|          Senegal|      United States|   40|
|          Moldova|      United States|    1|
+-----------------+-------------------+-----+
only showing top 10 rows



In [25]:
# In Python
(df.write.format("json")
    .mode("overwrite")
    .option("compression", "snappy")
    .save("/tmp/data/json/df_json"))

In [26]:
# In Python
file = "/Users/hyunjun/vscode/Spark/databricks-datasets/learning-spark-v2/flights/summary-data/csv/*"
schema = "DEST_COUNTRY_NAME STRING, ORIGIN_COUNTRY_NAME STRING, count INT"
df = (spark.read.format("csv")
    .option("header", "true")# Exit if any errors .option("nullValue", "")
    .schema(schema)# Replace any null data field with quotes 
    .option("mode", "FAILFAST")
    .load(file))

In [27]:
# -- In SQL
spark.sql("""CREATE OR REPLACE TEMPORARY VIEW us_delay_flights_tbl USING csv
OPTIONS (
path "/Users/hyunjun/vscode/Spark/databricks-datasets/learning-spark-v2/flights/summary-data/csv/*", header "true",
inferSchema "true",
mode "FAILFAST"
)""")

DataFrame[]

In [28]:
# // In Scala/Python
spark.sql("SELECT * FROM us_delay_flights_tbl").show(10)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|    1|
|    United States|            Ireland|  264|
|    United States|              India|   69|
|            Egypt|      United States|   24|
|Equatorial Guinea|      United States|    1|
|    United States|          Singapore|   25|
|    United States|            Grenada|   54|
|       Costa Rica|      United States|  477|
|          Senegal|      United States|   29|
|    United States|   Marshall Islands|   44|
+-----------------+-------------------+-----+
only showing top 10 rows



In [29]:
# In Python
df.write.format("csv").mode("overwrite").save("/tmp/data/csv/df_csv")

### Avro 오류로 인한 일시 중지...

In [30]:
# from pyspark.sql import SparkSession
# spark = SparkSession.builder.appName("myApp").getOrCreate()

# df = spark.read.format("avro").load("/Users/hyunjun/vscode/Spark/databricks-datasets/learning-spark-v2/flights/summary-data/avro/*")
# df.show(truncate=False)

In [31]:
# In Python
file = "/Users/hyunjun/vscode/Spark/databricks-datasets/learning-spark-v2/flights/summary-data/orc/*"
df = spark.read.format("orc").option("path", file).load()
df.show(10, False)


+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|United States    |Romania            |1    |
|United States    |Ireland            |264  |
|United States    |India              |69   |
|Egypt            |United States      |24   |
|Equatorial Guinea|United States      |1    |
|United States    |Singapore          |25   |
|United States    |Grenada            |54   |
|Costa Rica       |United States      |477  |
|Senegal          |United States      |29   |
|United States    |Marshall Islands   |44   |
+-----------------+-------------------+-----+
only showing top 10 rows



In [32]:
# -- In SQL
spark.sql("""CREATE OR REPLACE TEMPORARY VIEW us_delay_flights_tbl USING orc
OPTIONS (
path "/Users/hyunjun/vscode/Spark/databricks-datasets/learning-spark-v2/flights/summary-data/orc/*"
)"""
)

DataFrame[]

In [33]:
# // In Scala/Python
spark.sql("SELECT * FROM us_delay_flights_tbl").show()

+--------------------+-------------------+-----+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
|       United States|            Romania|    1|
|       United States|            Ireland|  264|
|       United States|              India|   69|
|               Egypt|      United States|   24|
|   Equatorial Guinea|      United States|    1|
|       United States|          Singapore|   25|
|       United States|            Grenada|   54|
|          Costa Rica|      United States|  477|
|             Senegal|      United States|   29|
|       United States|   Marshall Islands|   44|
|              Guyana|      United States|   17|
|       United States|       Sint Maarten|   53|
|               Malta|      United States|    1|
|             Bolivia|      United States|   46|
|            Anguilla|      United States|   21|
|Turks and Caicos ...|      United States|  136|
|       United States|        Afghanistan|    2|
|Saint Vincent and..

In [34]:
    # In Python
(df.write.format("orc")
    .mode("overwrite")
    .option("compression", "snappy")
    .save("/tmp/data/orc/flights_orc"))

In [35]:
# In Python
from pyspark.ml import image
image_dir = "/Users/hyunjun/vscode/Spark/databricks-datasets/learning-spark-v2/cctvVideos/train_images/"
images_df = spark.read.format("image").load(image_dir)
images_df.printSchema()

root
 |-- image: struct (nullable = true)
 |    |-- origin: string (nullable = true)
 |    |-- height: integer (nullable = true)
 |    |-- width: integer (nullable = true)
 |    |-- nChannels: integer (nullable = true)
 |    |-- mode: integer (nullable = true)
 |    |-- data: binary (nullable = true)
 |-- label: integer (nullable = true)



In [36]:
images_df.select("image.height", "image.width", "image.nChannels", "image.mode",
      "label").show(5, truncate=False)

+------+-----+---------+----+-----+
|height|width|nChannels|mode|label|
+------+-----+---------+----+-----+
|288   |384  |3        |16  |0    |
|288   |384  |3        |16  |1    |
|288   |384  |3        |16  |0    |
|288   |384  |3        |16  |0    |
|288   |384  |3        |16  |0    |
+------+-----+---------+----+-----+
only showing top 5 rows



In [37]:
# In Python
path = "/Users/hyunjun/vscode/Spark/databricks-datasets/learning-spark-v2/cctvVideos/train_images/"
binary_files_df = (spark.read.format("binaryFile")
    .option("pathGlobFilter", "*.jpg")
    .load(path))
binary_files_df.show(5)

+--------------------+-------------------+------+--------------------+-----+
|                path|   modificationTime|length|             content|label|
+--------------------+-------------------+------+--------------------+-----+
|file:/Users/hyunj...|2021-12-13 06:20:38| 55037|[FF D8 FF E0 00 1...|    0|
|file:/Users/hyunj...|2021-12-13 06:20:38| 54634|[FF D8 FF E0 00 1...|    1|
|file:/Users/hyunj...|2021-12-13 06:20:38| 54624|[FF D8 FF E0 00 1...|    0|
|file:/Users/hyunj...|2021-12-13 06:20:38| 54505|[FF D8 FF E0 00 1...|    0|
|file:/Users/hyunj...|2021-12-13 06:20:38| 54475|[FF D8 FF E0 00 1...|    0|
+--------------------+-------------------+------+--------------------+-----+
only showing top 5 rows



In [38]:
# In Python
binary_files_df = (spark.read.format("binaryFile")
    .option("pathGlobFilter", "*.jpg")
    .option("recursiveFileLookup", "true")
    .load(path))
binary_files_df.show(5)

+--------------------+-------------------+------+--------------------+
|                path|   modificationTime|length|             content|
+--------------------+-------------------+------+--------------------+
|file:/Users/hyunj...|2021-12-13 06:20:38| 55037|[FF D8 FF E0 00 1...|
|file:/Users/hyunj...|2021-12-13 06:20:38| 54634|[FF D8 FF E0 00 1...|
|file:/Users/hyunj...|2021-12-13 06:20:38| 54624|[FF D8 FF E0 00 1...|
|file:/Users/hyunj...|2021-12-13 06:20:38| 54505|[FF D8 FF E0 00 1...|
|file:/Users/hyunj...|2021-12-13 06:20:38| 54475|[FF D8 FF E0 00 1...|
+--------------------+-------------------+------+--------------------+
only showing top 5 rows

